In [1]:
import logging
import os
import tempfile
from abc import ABC, abstractmethod
from copy import deepcopy
from pathlib import Path
from typing import Type

import numpy as np
from ConfigSpace import Configuration, ConfigurationSpace
from smac import AlgorithmConfigurationFacade, Scenario
from smac.runhistory.dataclasses import TrialValue

from src.constant import DATA_DIR, TEMP_DIR, MAX_WORKERS
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.configuration_space.LKH import CONFIGURATION_SPACE

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [5]:
0.5 * 3600 / 10

180.0

In [3]:
t_c = 100
t_v = 100
K = 2
n = 2
solver_class = TSP_LKH_Solver

experiment = GlobalExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    solver_class=solver_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-12 22:36:43] INFO      [GLOBAL] Start!
[2024-11-12 22:36:43] INFO      Attempt 1/2
[2024-11-12 22:36:43] DEBUG     SMAC configuration, time: [100. 100.]
[2024-11-12 22:36:44] DEBUG     (0, 0) fn submitted
[2024-11-12 22:36:44] DEBUG     (0, 1) fn submitted
[2024-11-12 22:36:44] DEBUG     (1, 0) fn submitted
[2024-11-12 22:36:44] DEBUG     (1, 1) fn submitted
[2024-11-12 22:36:44] DEBUG     (2, 0) fn submitted
[2024-11-12 22:36:44] DEBUG     (2, 1) fn submitted
[2024-11-12 22:36:44] DEBUG     (3, 0) fn submitted
[2024-11-12 22:36:44] DEBUG     (3, 1) fn submitted
[2024-11-12 22:36:44] DEBUG     (4, 0) fn submitted
[2024-11-12 22:36:44] DEBUG     (4, 1) fn submitted
[2024-11-12 22:36:44] DEBUG     (0, 0) result
[2024-11-12 22:36:47] DEBUG     (0, 1) result
[2024-11-12 22:36:47] DEBUG     (1, 0) result
[2024-11-12 22:36:47] DEBUG     (1, 1) result
[2024-11-12 22:36:47] DEBUG     (2, 0) result
[2024-11-12 22:36:55] DEBUG     (2, 1) result
[2024-11-12 22:36:56] DEBUG     (3, 0) res

In [5]:
remaining_time = np.ones(shape=(K, )) * np.inf
best_portfolio.evaluate(test_instances, remaining_time, "test")

[2024-11-12 20:24:46] DEBUG     (0, 0) fn submitted
[2024-11-12 20:24:46] DEBUG     (0, 1) fn submitted
[2024-11-12 20:24:46] DEBUG     (0, 2) fn submitted
[2024-11-12 20:24:46] DEBUG     (0, 3) fn submitted
[2024-11-12 20:24:46] DEBUG     (1, 0) fn submitted
[2024-11-12 20:24:46] DEBUG     (1, 1) fn submitted
[2024-11-12 20:24:46] DEBUG     (1, 2) fn submitted
[2024-11-12 20:24:46] DEBUG     (1, 3) fn submitted
[2024-11-12 20:24:46] DEBUG     (2, 0) fn submitted
[2024-11-12 20:24:46] DEBUG     (2, 1) fn submitted
[2024-11-12 20:24:46] DEBUG     (2, 2) fn submitted
[2024-11-12 20:24:46] DEBUG     (2, 3) fn submitted
[2024-11-12 20:24:46] DEBUG     (3, 0) fn submitted
[2024-11-12 20:24:46] DEBUG     (3, 1) fn submitted
[2024-11-12 20:24:46] DEBUG     (3, 2) fn submitted
[2024-11-12 20:24:46] DEBUG     (3, 3) fn submitted
[2024-11-12 20:24:46] DEBUG     (4, 0) fn submitted
[2024-11-12 20:24:46] DEBUG     (4, 1) fn submitted
[2024-11-12 20:24:46] DEBUG     (4, 2) fn submitted
[2024-11-12 

1.0032127659574468

In [ ]:
# t_c = 100
# t_v = 100
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver

# experiment = ParhydraExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
import pandas as pd
pd.options.display.max_columns = 999

In [6]:
from src.database import db_connect
conn = db_connect()

In [ ]:
import sqlite3
conn = sqlite3.connect("database/2024_11_12_18_47_05.db")

In [3]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,50,0,NO,ALPHA,0,QUADRANT,YES,YES,1,WALK,1.000000,1,0,2147483647,5,5,1,0,50,YES,YES,0,YES
1,ul2HyyhdCxw5So7HCmczWDJgdsnTxYkoDrD7kcfe6ho=,57,0,YES,NEAREST-NEIGHBOR,7,QUADRANT,NO,NO,4,WALK,0.163349,3,4,754939590,5,5,5,3,65,NO,NO,5,NO
2,0ALcQOe-4JHD784MSTlc6RtbaT0FwhuvFExpZ4AEUG0=,57,0,YES,QUADRANT,3,QUADRANT,YES,YES,3,WALK,0.674050,5,5,1264675222,3,3,4,0,52,NO,NO,4,YES
3,GxsSEf4us1jJbqSnVsstzMNmi8t4IJMFOTQPnwT03rg=,51,0,NO,DELAUNAY,5,QUADRANT,NO,YES,2,BORUVKA,0.105218,2,5,1771838878,7,2,0,5,18,NO,YES,3,YES
4,fhm15vFZ9fG1S5OXPxevrWZJ7Cmf88JC35vFI1DwxkE=,58,0,NO,DELAUNAY,10,QUADRANT,NO,NO,4,WALK,0.332648,2,0,2091695967,6,6,2,1,60,YES,YES,3,NO
5,6dwko0JtGV1_8ff_KJMsvF16HRYTI48rDwCksEQpGBc=,56,0,YES,DELAUNAY,4,QUADRANT,YES,YES,4,WALK,0.527470,5,5,47329070,7,5,5,4,63,NO,YES,4,NO
6,oBbAyb3wEv3mdzx5eOJLib9QZ-Typuvgzy8VxGsUed8=,55,1,NO,QUADRANT,3,QUADRANT,YES,NO,5,BORUVKA,0.732126,5,4,1626134851,5,6,5,3,39,NO,YES,4,YES
7,pw4aemHJDtYF7jxGQsOGtAK9MOMV40sH-nkceE5swfU=,55,1,YES,QUADRANT,7,QUADRANT,NO,NO,3,GREEDY,0.931079,0,4,77296605,5,5,4,3,17,YES,YES,2,YES
8,IF-1xd7KxsJJ9-6Rh7Vl5jUfGWLYkpBMiyoOYZJp6I0=,57,0,NO,ALPHA,9,QUADRANT,YES,NO,2,SIERPINSKI,0.257109,0,5,2055513882,9,6,3,5,43,NO,NO,6,YES
9,EviY12KOQ2U9pp8BmbwShA95rjuYMievmr-fiics834=,42,1,YES,NEAREST-NEIGHBOR,10,QUADRANT,YES,NO,2,WALK,0.293864,3,0,1273806405,6,3,1,4,23,NO,YES,6,YES


In [4]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp
2,data/TSP/CEPS_benchmark/grid/33.tsp
3,data/TSP/CEPS_benchmark/rotation/45.tsp
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp
...,...
495,data/TSP/CEPS_benchmark/implosion/23.tsp
496,data/TSP/CEPS_benchmark/expansion/42.tsp
497,data/TSP/CEPS_benchmark/compression/17.tsp
498,data/TSP/CEPS_benchmark/cluster/47.tsp


In [5]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")

In [6]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_

solver_id,5ypBdP0XbjUP7j1a1-qHdW40TkZRIwOR4T9d7j2uS1c=,QyzjdpE6OiXgBRReHzJS9kyGDeCWQFOF49lr7nfA2Qc=,jgfHl8dW-GnRRX7ZBPl1o52rgqUzFJdixzAKLru8Npc=,wKeN4SEQhwyV-EG3qN6rJvh0NY1wNsUTRW-JpAnm3KQ=,best
instance_id,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,1.67,0.98,100.0,1.01,1
data/TSP/CEPS_benchmark/cluster/01.tsp,1.85,1.18,100.0,1.01,3
data/TSP/CEPS_benchmark/cluster/02.tsp,2.83,1.21,100.0,1.00,3
data/TSP/CEPS_benchmark/cluster/03.tsp,1.86,1.20,100.0,1.05,3
data/TSP/CEPS_benchmark/cluster/04.tsp,1.82,1.00,100.0,1.07,1
...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,1.84,1.19,100.0,1.00,3
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,1.66,0.93,100.0,0.98,1
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,1.72,0.94,100.0,0.97,1


In [ ]:
config = df_solvers.loc[df_solvers["id"] == "IF-1xd7KxsJJ9-6Rh7Vl5jUfGWLYkpBMiyoOYZJp6I0="].drop(columns=["id"]).iloc[0].to_dict()
config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
solver = TSP_LKH_Solver(config=config)

instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/expansion/21.tsp", optimum=10282268.0)

solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index